In [3]:
%reset -f
import glob, os, sys, io
sys.path.insert(1, os.path.join(sys.path[0], '..'))
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing
from sklearn.model_selection import train_test_split
import keras

from wavhandler import *
from utils import *

import logging
logger = logging.getLogger()
logger.propagate = False
logger.setLevel(logging.ERROR)
np.random.seed(0)
import seaborn as sns
sns.set()

In [4]:
X,y = get_data(target_names=all_6, nr_signals=20000, only_names=False)

100%|██████████| 6/6 [00:21<00:00,  3.57s/it]


In [5]:
def transform_data(X):
    from scipy import signal
    from tqdm import tqdm
    # transform the data
    XX = np.zeros((X.shape[0],4097)).astype("float32")   # allocate space
    for i in tqdm(range(X.shape[0])):
#         XX[i] = 10*np.log10(signal.welch(X[i], fs=F_S, window='hanning', nperseg=256, noverlap=128+64)[1])
#         XX[i] = power_spectral_density(X[i], only_powers=True)
#         sig = butter_bandpass_filter(data=X[i], lowcut=L_CUTOFF, highcut=H_CUTOFF, fs=F_S, order=B_ORDER)
        XX[i] = 10*np.log10(signal.welch(X[i], fs=F_S, window='hanning', nfft=8192, noverlap=None)[1])
    return XX
X = transform_data(X)

100%|██████████| 120906/120906 [11:47<00:00, 170.85it/s]


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    shuffle=True, 
                                                    random_state=0)

In [7]:
# Convert label to onehot
y_train = keras.utils.to_categorical(y_train, num_classes=6)
y_test = keras.utils.to_categorical(y_test, num_classes=6)

X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

# Build the Neural Network
model = Sequential()

model.add(Conv1D(16, 3, activation='relu', input_shape=(4097, 1)))
model.add(Conv1D(16, 3, activation='relu'))
model.add(BatchNormalization())

model.add(Conv1D(32, 3, activation='relu'))
model.add(Conv1D(32, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Conv1D(64, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(256, 3, activation='relu'))
model.add(Conv1D(256, 3, activation='relu'))
model.add(BatchNormalization())
model.add(GlobalAveragePooling1D())

model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [13]:
model_name = 'deep_1_PSD'
top_weights_path = './data/model_' + str(model_name) + '.h5'

In [14]:
callbacks_list = [ModelCheckpoint(top_weights_path, monitor = 'val_acc', 
                                  verbose = 1, 
                                  save_best_only = True, 
                                  save_weights_only = True),
                EarlyStopping(monitor = 'val_acc', 
                              patience = 6, 
                              verbose = 1),
                ReduceLROnPlateau(monitor = 'val_acc', 
                                  factor = 0.1, 
                                  patience = 3, 
                                  verbose = 1),
                CSVLogger('model_' + str(model_name) + '.log')]

In [15]:
model.fit(X_train, y_train, batch_size=128, epochs=100, validation_data = [X_test, y_test], callbacks = callbacks_list)


Train on 96724 samples, validate on 24182 samples
Epoch 1/100
96640/96724 [============================>.] - ETA: 0s - loss: 1.2472 - acc: 0.5224
Epoch 00001: val_acc improved from -inf to 0.16781, saving model to ./data/model_deep_1_PSD.h5
96724/96724 [==============================] - 492s 5ms/step - loss: 1.2472 - acc: 0.5224 - val_loss: 11.9265 - val_acc: 0.1678
Epoch 2/100
96640/96724 [============================>.] - ETA: 0s - loss: 1.1510 - acc: 0.5641
Epoch 00002: val_acc did not improve
96724/96724 [==============================] - 475s 5ms/step - loss: 1.1510 - acc: 0.5641 - val_loss: 13.6024 - val_acc: 0.1512
Epoch 3/100
96640/96724 [============================>.] - ETA: 0s - loss: 1.1122 - acc: 0.5810
Epoch 00003: val_acc improved from 0.16781 to 0.22707, saving model to ./data/model_deep_1_PSD.h5
96724/96724 [==============================] - 482s 5ms/step - loss: 1.1121 - acc: 0.5811 - val_loss: 11.0666 - val_acc: 0.2271
Epoch 4/100
96640/96724 [=======================

KeyboardInterrupt: 

In [ ]:
model.load_weights(top_weights_path)
loss, acc = model.evaluate(X_test, y_test, batch_size=16)

#print('loss', loss)
print('Test accuracy:', acc)